The [dataset](https://jmcauley.ucsd.edu/data/amazon_v2/index.html) is metadata for Magazine Subscriptions category

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules
DIR_PATH = ""

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/dirve")
    DIR_PATH = r"/content/dirve/MyDrive/研究所/Data/dblp/"
    sys.path.append('/content/dirve/MyDrive/Colab Notebooks/package')
else:
    DIR_PATH = r"D:\\論文實驗\\data\\dblp\\"
    sys.path.append('D:\\論文實驗\\package')
    sys.path.append("D:\\論文實驗\\env\\Lib\\site-packages")

In [12]:
import pandas as pd
from utils import preprocessingText

DIR_PATH = r"D:\\論文實驗\\data\\amazon\\"
DATASET = r"meta_Software.json"

def checkAttr():
    '''
        check whether ASIN of the item is existing.
        
        Result: 
            all of the items exist ASIN of the properties.
    '''
    with open(DIR_PATH + DATASET, "r") as f:
        data = pd.read_json(f, orient="records", typ="series", lines=True)
    
        for item in data:
            if not all(key in item for key in ["asin", "also_view", "also_buy", "category", "price"]):
                print(item)

def reduceAttr(output_file):
    '''
        保留需要的商品屬性, asin, also_view, also_buy, category, price.
        also_view和also_buy有多個以上時, 用空白隔開
    '''
    
    with open(DIR_PATH + DATASET, "r") as f:
        data = pd.read_json(f, orient="records", typ="series", lines=True, dtype={"category":list})

        output = open(DIR_PATH + output_file, "w")
        for item in data:
            also_view = " ".join(item["also_view"])
            also_buy = " ".join(item["also_buy"])
            category = " ".join([preprocessingText(string) for string in item["category"]])
            price = item["price"][1:] if item["price"] != "" and item["price"][0] == "$" else ""

            if also_view and also_buy and category and price:
                row = "{asin},{also_view},{also_buy},{category},{price}".format(
                    asin = item["asin"], 
                    also_view = also_view, 
                    also_buy = also_buy, 
                    category = category, 
                    price = price)
                output.write(row + "\n")
        output.close()
checkAttr()
reduceAttr("extraction.csv")

### 需要保留的商品屬性
- asin
- also_view
- also_buy
- category
- price

In [7]:
with open(DIR_PATH + DATASET, "r") as f:
    data = pd.read_json(f, orient="records", typ="series", lines=True)
    count = 0
    for item in data:
        if item["asin"] == "0078605407":
            count += 1
            also_view = " ".join(item["also_view"])
            also_buy = " ".join(item["also_buy"])
            category = " ".join([preprocessingText(string) for string in item["category"]])
            price = item["price"][1:] if item["price"] != "" and item["price"][0] == "$" else ""
            
            print("{asin},{also_view},{also_buy},{category},{price}".format(
                    asin = item["asin"], 
                    also_view = also_view, 
                    also_buy = also_buy, 
                    category = category, 
                    price = str(price)))
#print("{0}/{1}".format(count, data.size))

0078605407,0078769736 0078769744 0078605490 0078605423 0078271495 007876971X 0078742773,0078605490,software education  amp  reference languages,66.89


In [14]:
with open(DIR_PATH + "extraction.csv", "r") as f:
    count = 0
    for line in f:
        count = count + 1
    print(count)

855
